# Mini-Project I
During this project, we will practice handling of complex lists and dictionaries in Python. Plus, we will learn how to work with API documentation. Don't be afraid to search for the information in the [**documentation**](https://api.tfl.gov.uk/swagger/ui/index.html?url=/swagger/docs/v1#!/AccidentStats/AccidentStats_Get).

Let's go to the tasks, we have some parsing to do :)!!

In [ ]:
import requests
import os
import json

In [ ]:
# import packages we need (remember what packages we used yesterday during the API session)
# IMPORT HERE

## Demo

In [ ]:
# don't forget package os
app_id = os.environ["<your app id name>"]
app_key = os.environ["<your app key name>"]
url_append = f'?app_id={app_id}&app_key={app_key}' 

In [ ]:
print(url_append)

In [27]:
# URL
url = "https://api.tfl.gov.uk/AirQuality"

In [28]:
# We send the request to the API
# NOTE: if you don't have your APP_KEY, run this without the url_append
# res = requests.get(url+url_append)
res = requests.get(url)

In [4]:
# We can check if the request was successful
res.status_code

200

In [65]:
# We can export the information that was returned using method .json()
res.json()


{'$id': '1',
 '$type': 'Tfl.Api.Presentation.Entities.LondonAirForecast, Tfl.Api.Presentation.Entities',
 'updatePeriod': 'hourly',
 'updateFrequency': '1',
 'forecastURL': 'http://londonair.org.uk/forecast',
 'disclaimerText': 'This forecast is intended to provide information on expected pollution levels in areas of significant public exposure. It may not apply in very specific locations close to unusually strong or short-lived local sources of pollution.',
 'currentForecast': [{'$id': '2',
   '$type': 'Tfl.Api.Presentation.Entities.CurrentForecast, Tfl.Api.Presentation.Entities',
   'forecastType': 'Current',
   'forecastID': '33381',
   'forecastBand': 'Low',
   'forecastSummary': 'Low air pollution forecast valid from Wednesday 15 September to end of Wednesday 15 September GMT',
   'nO2Band': 'Low',
   'o3Band': 'Low',
   'pM10Band': 'Low',
   'pM25Band': 'Low',
   'sO2Band': 'Low',
   'forecastText': 'The North Sea air flow that started to affect Greater London on Tuesday evening 

In [280]:
def get_tfl_search_result(search):
    """returns api result in json form transport for london api - given the end url for the search"""
    base = "https://api.tfl.gov.uk/"
    url = base + search
    res = requests.get(url)
    if res.status_code != 200:
        print("It's not a 200 - better check it out!")
    return(res.json())
    

## Task
Parse the dictionary and print the AirQuality predictions for tomorrow

In [281]:
# url = "https://api.tfl.gov.uk/AirQuality"

result = get_tfl_search_result("/AirQuality")

print(result['currentForecast'][1]['forecastSummary'], '\n')
print(result['currentForecast'][1]['forecastText'])


Low air pollution forecast valid from Saturday 18 September to end of Saturday 18 September GMT 

Partly sunny with showers on Saturday.&lt;br/&gt;&lt;br/&gt;A light south or south easterly breeze is expected. This airflow will have travelled over the near continent and at times will have passed over the Paris region. It&#39;s likely to contain some particulates and as a result both PM10 &amp; PM2.5 particulates may increase during the day. It&#39;s not expected currently that particulate concentrations will reach the &#39;moderate&#39; pollution threshold.&lt;br/&gt;&lt;br/&gt;Air pollution is expected to remain &#39;Low&#39; during the forecast period for the following pollutants.&lt;br/&gt;&lt;br/&gt;Nitrogen Dioxide&lt;br/&gt;Sulphur Dioxide&lt;br/&gt;Ozone&lt;br/&gt;PM10 Particulates&lt;br/&gt;PM2.5 Particulates&lt;br/&gt;


## Task
What are the different modes of transport which are operated by Transfer for London? How many of modes do they have?

Print the list with different modes of transport, plus their count. Example output:
```
[bus, cable-car,.....]
Number of different modes of transport is: xyz
```

We need to search the documentation for correct request.

In [287]:
# url = "https://api.tfl.gov.uk/Line/Meta/Modes"

search = "/Line/Meta/Modes"
result = get_tfl_search_result(search)

# Response was a single layer list - each item in was for a different mode of transportation
# Sorted through to find only those run by Transfer:

modes = []
for i in range(len(response)):
# for i,mode in enumerate(response):
    if response[i]['isTflService']:
        modes.append(response[i]['modeName'])

print(modes)
print(f'{len(modes)} modes of transport operated by Transfer.')


['bus', 'cable-car', 'cycle-hire', 'dlr', 'overground', 'replacement-bus', 'river-bus', 'river-tour', 'tflrail', 'tram', 'tube']
11 modes of transport operated by Transfer.


## Task
How many BikePoints in London are operated by Transfor for London? How many docks are in **all** BikePoints? There is the information for empty and full docks for each BikePoint.

In [288]:
# sum ALL docks for ALL bikePoints (want both empty and full docks)

# url = "https://api.tfl.gov.uk/BikePoint/"
search = "/BikePoint/"
result = get_tfl_search_result(search)

total_docks_for_bike_points = []

all_bike_points = []
total_empty = []
total_filled = []

for i in range(len(result)):
    all_bike_points.append([result[i]["id"]])
   
    for j in range (len(result[i]["additionalProperties"])):
        
        if result[i]["additionalProperties"][j]["key"] == "NbBikes":
            try:
                total_filled.append(int(result[i]["additionalProperties"][j]["value"]))
            except TypeError:
                total_filled.append(0)
                print('bike point data issues')

        elif result[i]["additionalProperties"][j]["key"] == "NbEmptyDocks":
            try:
                total_empty.append(int(result[i]["additionalProperties"][j]["value"]))
            except:
                total_empty.append(0)
                print('empty issues')
                                   
        elif result[i]["additionalProperties"][j]["key"] == "NbDocks":
            try:
                total_docks_for_bike_points.append(int(result[i]["additionalProperties"][j]["value"]))
            except TypeError:
                total_docks_for_bike_points.append(0)
                print('dock data issues')

                                   
print ("Total number of bike points:", len(all_bike_points))
print("Total number bike point docks:", sum(total_docks_for_bike_points), '\n')
print(f'empty docks: {sum(total_empty)}, filled docks: {sum(total_filled)}, broken docks: {sum(total_docks_for_bike_points) - sum(total_filled) -  sum(total_empty)}')


# checked response length to total_docks_for_bike_points length to ensure all dock info came over - it did
# print(len(total_docks_for_bike_points))


Total number of bike points: 786
Total number bike point docks: 20860 

empty docks: 11778, filled docks: 8104, broken docks: 978


## Task
How many tube and bus lines are in London? Print names of all tube lines.

In [271]:
# https://api.tfl.gov.uk/Line/Mode/{modes}/Route[?serviceTypes]

bus_search = "/Line/Mode/bus"
tube_search = "/Line/Mode/tube"

bus_result = get_tfl_search_result(bus_search)
tube_result = get_tfl_search_result(tube_search)
    
print(f'There are {len(bus_result)} bus lines and {len(tube_result)} tube lines. \n' )
print(f'List of Tubes Line:')

for i in range(len(tube_result)):
    print(tube_result[i]["name"])


There are 677 bus lines and 11 tube lines. 

List of Tubes Line:
Bakerloo
Central
Circle
District
Hammersmith & City
Jubilee
Metropolitan
Northern
Piccadilly
Victoria
Waterloo & City


## Task
How many station has `victoria` line?

In [273]:
# line id from above problem found id='victora'
# had such a hard time finding the right URL for this one and had help locating it

search = "Line/victoria/StopPoints"
result = get_tfl_search_result(search)

stops = []
# for i, stop in enumerate(response):
for i in range(len(result)):
    stops.append(result[i]["commonName"])


print(f'There are {len(stops)} stations on the Victoria Line.')


There are 16 stations on the Victoria Line.


## Task
Plan the journey from Heathrow Airport to Tower Bridge using Bus and Tube? Which way is faster? Example output:
```
Planned duration:
Bus: x minutes
Tube: y minutes
```

We need to search the documentation for correct requests and parameters we need.

In [ ]:
https://api.tfl.gov.uk/Journey/JourneyResults/{from}/to/{to}[?via][&nationalSearch][&date][&time][&timeIs][&journeyPreference][&mode][&accessibilityPreference][&fromName][&toName][&viaName][&maxTransferMinutes][&maxWalkingMinutes][&walkingSpeed][&cyclePreference][&adjustment][&bikeProficiency][&alternativeCycle][&alternativeWalking][&applyHtmlMarkup][&useMultiModalCall][&walkingOptimization][&taxiOnlyTrip][&routeBetweenEntrances]

In [ ]:


# re.get('https://api.tfl.gov.uk/Journey/JourneyResults/51.4700%2C%20-0.4543/to/51.5055%2C%20-0.0754?mode={}'.format(m))
# lat long from stations

# parse through called match quality (0-1000)(how close locatin is to desired destination - use that ) Heathrow 

In [277]:
# https://api.tfl.gov.uk/Journey/JourneyResults/{from}/to/{to}[?via][&nationalSearch][&date][&time][&timeIs][&journeyPreference][&mode][&accessibilityPreference][&fromName][&toName][&viaName][&maxTransferMinutes][&maxWalkingMinutes][&walkingSpeed][&cyclePreference][&adjustment][&bikeProficiency][&alternativeCycle][&alternativeWalking][&applyHtmlMarkup][&useMultiModalCall][&walkingOptimization][&taxiOnlyTrip][&routeBetweenEntrances]
def get_journey(mode):
    search = "/Journey/JourneyResults/51.4700%2C%20-0.4543/to/51.5055%2C%20-0.0754?mode="
    result = get_tfl_search_result(search + mode)
 
    print(f'By {mode} it will take {result["journeys"][0]["duration"]} minutes. \n')
    print(f'Depart at: {result["journeys"][0]["startDateTime"]}')
    print(f'Arrive at: {result["journeys"][0]["arrivalDateTime"]} \n')
    
    instructions = []
    for i in range (len(result["journeys"][0]["legs"])):
        new_step = result["journeys"][0]["legs"][i]["instruction"]["summary"]
        instructions.append(new_step)
    
    print(f'By {mode} there are {len(instructions) - 1} transfers: \n')
    for i in range(len(instructions)):
        print(instructions[i])


In [291]:
get_journey("bus")

By bus it will take 102 minutes. 

Depart at: 2021-09-17T02:30:00
Arrive at: 2021-09-17T04:12:00 

By bus there are 3 transfers: 

Walk to Heathrow Central Bus Station
N9 bus to Trafalgar Square
N15 bus to The Tower Of London
Walk to 1 Tower Bridge, Tower Hamlets


In [292]:
get_journey("tube")

By tube it will take 319 minutes. 

Depart at: 2021-09-17T00:11:00
Arrive at: 2021-09-17T05:30:00 

By tube there are 6 transfers: 

Walk to Heathrow Terminals 2 & 3 Underground Station
Piccadilly line to Acton Town
District line to Ealing Broadway
Central line to White City
Walk to Wood Lane
Hammersmith & City line to Aldgate
Walk to 1 Tower Bridge, Tower Hamlets


In [290]:
get_journey('cycle-hire')

By cycle-hire it will take 102 minutes. 

Depart at: 2021-09-17T02:30:00
Arrive at: 2021-09-17T04:12:00 

By cycle-hire there are 3 transfers: 

Walk to Heathrow Central Bus Station
N9 bus to Trafalgar Square
N15 bus to The Tower Of London
Walk to 1 Tower Bridge, Tower Hamlets
